In [7]:
from docx import Document

In [8]:
# check if each reference in the references document is present in the paper document
reference_docx = 'P:/John/crossing-paper/barrel-references v2.docx'
ref_doc = Document(reference_docx)

paper_docx = 'P:/John/crossing-paper/barrel-paper v6.docx'
paper_doc = Document(paper_docx)

#disc_docx = 'P:/John/crossing-paper/barrel-discussion v6.docx'
#disc_doc = Document(disc_docx)


In [11]:
# for each reference, check that it is in the paper. Print the reference if it is not found.
n_not_found = 0
for para in ref_doc.paragraphs:
    if para.text:
        ref = para.text.split()[0].replace(',', '').strip()
        found = False
        for paper_para in paper_doc.paragraphs:
            if ref in paper_para.text:
                found = True
                break
        #if not found:
        #for disc_para in disc_doc.paragraphs:
        #    if ref in disc_para.text:
        #        #print(f"Found in discussion: '{ref}'")
        #        found = True
        #        break
        if not found:
            print(f"Not cited: '{para.text}'")
            n_not_found += 1

print(f"Total references not found: {n_not_found} of {len(ref_doc.paragraphs)}" )


Not cited: 'Lazic, S.E. The problem of pseudoreplication in neuroscientific studies: is it affecting your analysis?. BMC Neurosci 11, 5 (2010). https://doi.org/10.1186/1471-2202-11-5'
Not cited: 'Chelaru MI, Dragoi V. Asymmetric synaptic depression in cortical networks. Cereb Cortex. 2008 Apr;18(4):771-88.'
Not cited: 'Civillico EF, Contreras D. Comparison of responses to electrical stimulation and whisker deflection using two different voltage-sensitive dyes in mouse barrel cortex in vivo. J Membr Biol. 2005 Nov;208(2):171-82.'
Not cited: 'Sachdev RNS, Krause MR, Mazer JA. Surround suppression and sparse coding in visual and barrel cortices. Front. Neural Circuits, 04 July 2012;6.'
Not cited: 'Bruno RM, Simons DJ. Feedforward mechanisms of excitatory and inhibitory cortical receptive fields. J Neurosci. 2002 Dec 15;22(24):10966-75.'
Total references not found: 5 of 114


In [13]:

# look through the paper and discussion documents,
# parse any citations from the paragraph and 
# then check they are in references
# references in the format (Author, Year) or (Author and Author2, Year) or (Author et. al., Year)
#  or a concatenation of any of these (Author, Year; Author et. al., Year)
# there may be multiple citations in a single paragraph

n_not_found = 0

for para in paper_doc.paragraphs: # + disc_doc.paragraphs:
    # find all "(" and ")" pairs and parse the text between them
    start = 0
    while True:
        start = para.text.find('(', start)
        if start == -1:
            break
        end = para.text.find(')', start)
        if end == -1:
            break
        citation = para.text[start+1:end]
        # must have from 2 to 4 words in the citation
        if not (2 <= len(citation.split()) <= 4):
            start = end + 1
            continue
        # split by semicolon to get multiple citations
        citations = [c.strip() for c in citation.split(';')]
        for c in citations:
            # must contain a year, which is a 4-digit number, as the last part of the citation
            last_word = c.split()[-1]
            if not last_word.isdigit() or len(last_word) != 4:
                continue
            # check if this citation is in the reference document
            found = False
            # there is a match if each word other than "and", "et", or "al" is found in any of the reference paragraphs
            c_words = c.replace('and', '').replace(' et.', '').replace(' al.', '').split()
            # strip ., and any other punctuation from the words
            c_words = [word.strip('.,') for word in c_words if word.strip('.,')]
            if not c_words:
                continue
            # check each word in the citation
            found = all([any((word.strip('.,') in ref_para.text) for ref_para in ref_doc.paragraphs) for word in c_words])  # remove punctuation from words

            # if all words were found, then the citation is valid
            if not found:
                print(f"Not cited: '{c}' in paragraph: '{para.text}'")
                print("\t keywords contained: ", c_words)
                n_not_found += 1
        start = end + 1

print(f"Total citations not found: {n_not_found} of {len(paper_doc.paragraphs)}")


Total citations not found: 0 of 222


In [ ]:
# open a text file and check for duplicate references
text_file = 'C:/Users/jjudge3/Downloads/barrel-references.txt'
with open(text_file, 'r') as f:
    lines = f.readlines()

for i in range(10):
    
    lines = [line.strip() for line in lines if line.strip()]  # remove empty lines and strip whitespace
    # remove first word from each line
    lines = [line.split(' ', 1)[-1] for line in lines]
    unique_lines = set(lines)  # use a set to find unique lines
    if len(unique_lines) < len(lines):
        print("\n\n Trial", i)
        print(f"Found {len(lines) - len(unique_lines)} duplicate lines.")

        # print the duplicate lines
        duplicates = set([line for line in lines if lines.count(line) > 1])
        print("Duplicate lines:")
        for dup in duplicates:
            print(dup)